In [2]:
import torch
import torch.nn.functional as F

def resize_with_padding(image, target_size=640):
    # Convert to float32 if not already
    if image.dtype != torch.float32:
        image = image.float()

    # Get current dimensions
    height, width = image.shape[:2]

    # Calculate new dimensions while maintaining aspect ratio
    ratio = target_size / max(height, width)
    new_height = int(height * ratio)
    new_width = int(width * ratio)

    # Resize the image
    resized_image = F.interpolate(image.unsqueeze(0).permute(0, 3, 1, 2), size=(new_height, new_width), mode='bilinear', align_corners=False).squeeze(0).permute(1, 2, 0)

    # Calculate padding
    pad_height = target_size - new_height
    pad_width = target_size - new_width

    # Add padding
    padding = [
        pad_width // 2, pad_width - pad_width // 2,  # Left, Right
        pad_height // 2, pad_height - pad_height // 2  # Top, Bottom
    ]

    padded_image = F.pad(resized_image.permute(2, 0, 1), padding, mode='constant', value=0).permute(1, 2, 0)

    return padded_image

In [15]:
from PIL import Image

def resize_image_to_smaller_dimension(image_path, output_path, target_size):
    # Open an image file
    with Image.open(image_path) as img:
        # Get original dimensions
        original_width, original_height = img.size
        
        # Determine the scaling factor based on the larger dimension
        if original_width > original_height:
            scale_factor = target_size / original_width
        else:
            scale_factor = target_size / original_height
        
        # Calculate new dimensions
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)
        
        # Resize the image
        resized_img = img.resize((new_width, new_height))

        # Save the resized image
        resized_img.save(output_path)


In [ ]:
from PIL import Image

img = resize_image_to_smaller_dimension('/workspaces/ai-capstone-proj/examples/image_0000005.jpg', '/workspaces/ai-capstone-proj/examples/image_0000005_resize.jpg',  640)


In [19]:
from PIL import Image

def resize_and_pad_to_square(image_path, output_path, target_size):
    # Open an image file
    with Image.open(image_path) as img:
        # Get original dimensions
        original_width, original_height = img.size
        
        # Determine the scaling factor based on the larger dimension
        if original_width > original_height:
            scale_factor = target_size / original_width
        else:
            scale_factor = target_size / original_height
        
        # Calculate new dimensions
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)
        
        # Resize the image
        resized_img = img.resize((new_width, new_height))
        
        # Create a new square image with a black background
        square_img = Image.new("RGB", (target_size, target_size), (0, 0, 0))
        
        # Calculate position to center the resized image
        position = ((target_size - new_width) // 2, (target_size - new_height) // 2)
        
        # Paste the resized image onto the square image
        square_img.paste(resized_img, position)

        # Save the padded image
        square_img.save(output_path)

        # Return the padded image and the position
        return square_img, position

# Example usage
padded_image, position = resize_and_pad_to_square('/workspaces/ai-capstone-proj/examples/image_0000005.jpg', '/workspaces/ai-capstone-proj/examples/image_0000005_resize.jpg',  640)
print("Position of the image:", position)


Position of the image: (107, 0)


In [21]:
from PIL import Image

def resize_and_pad_to_square_yolo(image_path, output_path, target_size):
    # Open an image file
    with Image.open(image_path) as img:
        # Get original dimensions
        original_width, original_height = img.size
        
        # Determine the scaling factor based on the larger dimension
        if original_width > original_height:
            scale_factor = target_size / original_width
        else:
            scale_factor = target_size / original_height
        
        # Calculate new dimensions
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)
        
        # Resize the image
        resized_img = img.resize((new_width, new_height))
        
        # Create a new square image with a black background
        square_img = Image.new("RGB", (target_size, target_size), (0, 0, 0))
        
        # Calculate position to center the resized image
        position = ((target_size - new_width) // 2, (target_size - new_height) // 2)
        
        # Paste the resized image onto the square image
        square_img.paste(resized_img, position)

        # Save the padded image
        square_img.save(output_path)

        # Calculate YOLO format coordinates
        # Center of the bounding box
        center_x = (position[0] + new_width / 2) / target_size
        center_y = (position[1] + new_height / 2) / target_size
        
        # Width and height of the bounding box
        width = new_width / target_size
        height = new_height / target_size
        
        # Return the padded image and YOLO format coordinates
        return square_img, (center_x, center_y, width, height)

# Example usage
padded_image, yolo_coordinates = resize_and_pad_to_square_yolo('/workspaces/ai-capstone-proj/examples/image_0000005.jpg', '/workspaces/ai-capstone-proj/examples/image_0000005_resize.jpg',  640)
print("YOLO format coordinates:", yolo_coordinates)


YOLO format coordinates: (0.5, 0.5, 0.665625, 1.0)
